# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
K. Doi  ->  K. Doi  |  ['K. Doi']
N. Wang  ->  N. Wang  |  ['N. Wang']


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Reinarz  ->  Y. Reinarz  |  ['Y. Reinarz']
Arxiv has 68 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.12325


extracting tarball to tmp_2602.12325...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.13007
extracting tarball to tmp_2602.13007...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.13125
extracting tarball to tmp_2602.13125...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.13007/arXiV_version.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Atomic_data' from 'tmp_2602.13007/Atomic_data.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


 done.


L. Acuña  ->  L. Acuña  |  ['L. Acuña']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Reinarz  ->  Y. Reinarz  |  ['Y. Reinarz']


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.12325-b31b1b.svg)](https://arxiv.org/abs/2602.12325) | **Direct pathway to the Early Supermassive Black Holes: A Red Super-Eddington Quasar in a Massive Starburst Host at $z=7.2$**  |
|| Q. Fei, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2026-02-16*|
|*Comments*| *26 pages, 12 figures, 2 tables. Submitted to ApJ. Comments are welcome!*|
|**Abstract**|            We present a panchromatic optical-mm characterization of GNz7q, a recently identified X-ray weak, rapidly growing red quasar embedded within a dusty starburst galaxy at $z=7.1899$, using the full suite of JWST/NIRCam, NIRSpec, MIRI, and archival NOEMA observations. Our deep NIRSpec/G395M spectroscopy reveals unambiguous broad Balmer emission (FWHM $=2221\pm20$kms$^{-1}$), confirming a super-Eddington accreting black hole ($\lambda_{\rm Edd}=2.7\pm0.4$) with a mass of $\log(M_{\rm BH}/M_{\odot})=7.55\pm0.34$, using accretion-rate corrected BH mass estimators. After subtracting the point source, we robustly detect stellar emission from the host galaxy across multiple NIRCam and MIRI filters. Out joint morphological-spectral analysis yields a stellar mass of $\log (M_*/M_\odot)=10.5\pm0.4$ and an intense star formation rate of ${\rm SFR}=330\pm97\,M_\odot\,\rm yr^{-1}$, confirming the host as a massive, dusty starburst galaxy. We find that GNz7q lies on the local $M_{\rm BH}$-$M_*$ relation ($M_{\rm BH}/M_*\simeq 0.001$) and is well positioned to evolve into the locus of massive SDSS quasars with $\log (M_{\rm BH}/M_\odot)\approx 9$ and $M_*\approx 10^{11}\,M_\odot$ at $z\sim 6$, owing to its remarkably rapid growth in both the black hole and its host galaxy. This stands in stark contrast to many recently reported JWST AGN populations at similar redshifts, including the little red dots (LRDs), whose weak or undetected star formation makes it difficult for them to grow into the massive galaxies hosting SDSS-like quasars. These results suggest that GNz7q marks as a rare, pivotal phase of early BH-galaxy co-eolution, plausibly providing a crucial direct pathway to the supermassive black hole systems within the first billion years of the Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.12522-b31b1b.svg)](https://arxiv.org/abs/2602.12522) | **A Protoplanet Candidate in the PDS 66 Disk Indicated by Silicon Sulfide Isotopologues**  |
|| T. C. Yoshida, et al. -- incl., <mark>M. Benisty</mark>, <mark>K. Doi</mark> |
|*Appeared on*| *2026-02-16*|
|*Comments*| *16 pages, 8 figures, accepted for publication in ApJL*|
|**Abstract**|            Despite observational progress in planet formation, the stage in which planetesimals grow into planets remains poorly understood. During this phase, protoplanets may develop gaseous envelopes that are warmer than the surrounding disk gas, potentially providing observable signatures through molecules otherwise depleted in cold regions. In this Letter, we report the detection of the silicon sulfide isotopologues ${\rm ^{28}SiS}\ J{=}16{-}15$ and ${\rm ^{30}SiS}\ J{=}18{-}17$ in the protoplanetary disk around PDS 66 (MP Mus) at a significance of ${\sim}5{-}6\sigma$, using the Atacama Large Millimeter/submillimeter Array. These constitute the second and first detections of $\rm ^{28}SiS$ and $\rm ^{30}SiS$ in a protoplanetary disk, respectively. The emission appears as a compact source at $r \simeq 60$ au in the southwestern region of the disk, unresolved with a ${\sim}0.\!\!^{\prime\prime}5$ beam, and shows a velocity consistent with Keplerian rotation, suggesting a protoplanetary origin. By modeling the line fluxes, we constrain the emitting radius to ${\sim}0.5{-}4$ au and estimate an SiS mass of $10^{22}{-}10^{23}$ g, corresponding to at least ${\sim}10\%$ of the silicon contained in local dust grains. Because complete sublimation of a substantial fraction of dust grains by local processes is difficult to achieve, this result instead implies an accumulation of silicon from a larger region. We propose that a circumplanetary envelope surrounding a low-mass protoplanet, where pebble accretion and subsequent sublimation of grains may enhance gaseus silicon abundance with respect to observable dust grains around it, can account for the observed characteristics.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.13125-b31b1b.svg)](https://arxiv.org/abs/2602.13125) | **TIC65910228b: A single-transit discovery of a massive long-period warm Jupiter with TESS**  |
|| F. I. Rojas, et al. -- incl., <mark>L. Acuña</mark>, <mark>T. Henning</mark>, <mark>Y. Reinarz</mark> |
|*Appeared on*| *2026-02-16*|
|*Comments*| *11 pages, 11 figures*|
|**Abstract**|            Context. Warm Jupiters are excellent case studies for the investigation of giant planet internal structures and formation theories. However, the sample of long-period transiting giants is still small today for a better understanding of this population. Aims. Starting from a single transit found in the Transiting Exoplanet Survey Satellite (TESS) data, we confirm the planetary nature of the signal and measure its orbital parameters, mass, and radius. We put this system in the context of long-period giant transiting planets and analyzed the viability to sustain atmospheric or dynamical follow-up. Methods. We carried out a spectroscopic follow-up using FEROS and PLATOSpec to obtain precise radial velocities. We added a photometric follow-up with HATPI and Observatoire Moana to obtain a more precise estimate of the orbital period. We derived the orbital and physical parameters through a joint analysis of this data. Results. We report the discovery and characterization of TIC65910228b, a transiting warm Jupiter with a mass of $4.554 \pm 0.255$ $M_J$ and a radius of $1.088 \pm 0.061$ $R_J$, orbiting an evolved F-type star every $\sim 180.52$ days in an eccentric orbit ($e = 0.25 \pm 0.04$). Conclusions. This planet joins a still under-explored population of long-period ($P > 100$) massive ($M_p > 4$ $M_J$) transiting giant planets, being one of the few with a mild eccentricity. This target is a nice example of the potential of single-transit events to populate this region of the parameter space.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.12479-b31b1b.svg)](https://arxiv.org/abs/2602.12479) | **Odd Radio Circles Modeled by Shock-Bubble Interactions**  |
|| <mark>Y. Wang</mark>, S. Heinz |
|*Appeared on*| *2026-02-16*|
|*Comments*| **|
|**Abstract**|            The physical nature and origins of the newly discovered class of Odd Radio Circles (ORCs) remain unclear. We investigate a model whereby ORCs are synchrotron-emitting vortex rings formed by the Richtmyer-Meshkov instability (RMI) when a shock interacts with a low-density fossil radio lobe in the intergalactic medium using 3D magnetohydrodynamic simulations. These rings initially exhibit oscillatory behavior that damps over time. We implement a new method to model Inverse-Compton cooling and synchrotron cooling at high frequencies in a scale-free manner, enabling us to test a wide range of model parameters against the observational constraints. We find that shock strengths of Mach 2-4 are consistent with the data, as expected in accretion, merger-driven, or active galactic nuclei-driven shocks. We find that the initial size of the bubbles required to explain the rings ranges from 140 to 250 kpc, with initial energy in the bubble of order $10^{57}-10^{59}$ erg, consistent with fossil lobes inflated by moderately powerful radio galaxies. Derived ambient pressures and densities place ORCs in low density environments, such as the outskirts of galaxy groups with ages of order 70-200 Myr. Our synthetic radio maps match the polarization properties of ORC1 and predict a dependency of the tangential magnetic field angle on the aspect ratio of ORCs. A key distinguishing trait of the RMI-driven vortex ring model is that it does not require the ORC to be centered on its host galaxy and is therefore redshift agnostic.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.12677-b31b1b.svg)](https://arxiv.org/abs/2602.12677) | **Murriyang cryogenic phased array feed: spectral-line results and noise-reduction methods**  |
|| L. Staveley-Smith, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2026-02-16*|
|*Comments*| *20 pages, 18 figures, accepted by PASA (11 February 2026), abbreviated abstract*|
|**Abstract**|            Spectral-line results from a new cryogenic phased array feed (cryoPAF) on the Murriyang telescope at Parkes are presented. This array offers a significant improvement in field of view, aperture efficiency, bandwidth, chromaticity and survey speed compared with conventional horn-fed receivers. We demonstrate this with measurements of sky calibrators and observations of 21-cm neutral hydrogen (HI) in the LMC and the nearby galaxy NGC 6744. Within 0.3 deg of the optical axis, the ratio of system temperature to dish aperture efficiency is 25 K and the ratio with beam efficiency is 21 K (at 1.4 GHz). For the previously measured $T_{sys} = 17$ K, respective efficiency values 0.7 and 0.8 are derived. Our HI observational results are in good agreement with previous results, although detailed comparison with multibeam observations of the LMC suggests that the earlier observations may have missed an extended component of low-column-density gas ($8\times 10^{18}$ cm$^{-2}$). We use the cryoPAF zoom-band and wideband data to make a preliminary investigation of whether the large number of simultaneous beams (72) permits the use of novel data reduction methods to reduce the effects of foreground/background continuum contamination and RFI. We also investigate if these methods can better protect against signal loss for the detection of faint, extended cosmological signals such as HI intensity maps. Using robust higher-order singular value decomposition (SVD) techniques, we find encouraging results for the detection of both compact and extended sources, including challenging conditions with high RFI occupancy and significant sky continuum structure. Examples are shown that demonstrate that 3D SVD techniques offer a significant improvement in noise reduction and signal capture compared with more traditional layered 2D techniques.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.13007-b31b1b.svg)](https://arxiv.org/abs/2602.13007) | **Self-Consistent Direct Method for Chemical Abundances in High-z Galaxies with JWST**  |
|| K. Z. Arellano-Córdova, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2026-02-16*|
|*Comments*| *Under revision with MNRAS. 13 pages, 6 figures*|
|**Abstract**|            The unprecedented rest-frame UV and optical coverage provided by JWST enables simultaneous constraints on the electron density (n$_{\rm e}$) and temperature (T$_{\rm e}$) of ionized gas in galaxies at z>5. We present a self-consistent direct method based on multiple OIII]1661,66) and [OIII] ($\lambda$4363, and $\lambda$5007) transitions to characterize the physical conditions of the high-ionization zone. This new approach is insensitive to a wide range of n$_{\rm e}$ due to the high critical densities of the OIII] and [OIII] transitions. Applying this technique to six galaxies at z=5-9, we find electron densities up to n$_{\rm e}$$\sim 3\times 10^{5}$ cm$^{-3}$ and temperatures of T$_{\rm e}$ $\sim 20,000$ K in systems at $z>6$. Accounting for these self-consistent densities changes the derived T$_{\rm e}$ and modifies the inferred metallicities by up to 0.29 dex relative to previous estimates. We discuss the reported N/O overabundances in the high-$z$ galaxies from our sample, which arise entirely from the high N$^{3+}$/H$^{+}$ values inferred from NIV] lines. We point out that a T$_{\rm e}$-stratification, in which the N$^{3+}$ zone has a slightly higher T$_{\rm e}$ than T$_{\rm e}$([OIII]), could substantially reduce the inferred N/O. Quantitatively, if T$_{\rm e}$(N$^{3+}$) were 10\% higher than T$_{\rm e}$([OIII]), this could induce a systematic overestimation of N$^{3+}$/O$^{2+}$ of nearly 50\%. Classical N/O diagnostics such as N$^{+}$/O$^{+}$, due to their critical densities, can significantly impact the inferred N/O abundance in the presence of high-density gas, whereas N$^{2+}$/O$^{2+}$ place these galaxies closer to $z\sim0$ systems in the N/O-O/H plane. Future JWST programs with larger and more diverse samples will be essential to test the universality and robustness of these results.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.12325/./figures/01_MIRI_LRS_fit.png', 'tmp_2602.12325/./figures/03_SED_fitting.png', 'tmp_2602.12325/./figures/PaLineMap.png', 'tmp_2602.12325/./figures/Paa_fit_1Gauss_zoom.png']
copying  tmp_2602.12325/./figures/01_MIRI_LRS_fit.png to _build/html/
copying  tmp_2602.12325/./figures/03_SED_fitting.png to _build/html/
copying  tmp_2602.12325/./figures/PaLineMap.png to _build/html/
copying  tmp_2602.12325/./figures/Paa_fit_1Gauss_zoom.png to _build/html/
exported in  _build/html/2602.12325.md
    + _build/html/tmp_2602.12325/./figures/01_MIRI_LRS_fit.png
    + _build/html/tmp_2602.12325/./figures/03_SED_fitting.png
    + _build/html/tmp_2602.12325/./figures/PaLineMap.png
    + _build/html/tmp_2602.12325/./figures/Paa_fit_1Gauss_zoom.png
found figures ['tmp_2602.12522/./mom0.png', 'tmp_2602.12522/./spectra_CDMS.png', 'tmp_2602.12522/./NRM_2.png']
copying  tmp_2602.12522/./mom0.png to _build/html/
copying  tmp_2602.12522/./spectra_CDMS.png to _build/html/
copying  t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\cii}{[C \textsc{ii}]}$
$\newcommand{\feii}{Fe \textsc{ii}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\sersic}{Sérsic}$
$\newcommand{\targ}{GNz7q}$
$\newcommand{\galfits}{\textsc{GalfitS}}$
$\newcommand{\cigale}{\texttt{cigale}}$
$\newcommand{\tcr}{\textcolor{red}}$
$\newcommand{\tcb}{\textcolor{blue}}$
$\newcommand{\tcm}{\textcolor{magenta}}$
$\newcommand{\tcg}{\textcolor{cyan}}$
$\newcommand{\tck}{\textcolor{black}}$
$\newcommand{\PN}{\textcolor{green}}$</div>



<div id="title">

# Direct pathway to the Early Supermassive Black Holes: \ A Red Super-Eddington Quasar in a Massive Starburst Host at $\boldmath$ $z=7.2$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.12325-b31b1b.svg)](https://arxiv.org/abs/2602.12325)<mark>Appeared on: 2026-02-16</mark> -  _26 pages, 12 figures, 2 tables. Submitted to ApJ. Comments are welcome!_

</div>
<div id="authors">

Q. Fei, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a panchromatic optical--mm characterization of $\targ$ , a recently identified X-ray weak, rapidly growing red quasar embedded within a dusty starburst galaxy at $z=7.1899$ , using the full suite of JWST/NIRCam, NIRSpec, MIRI, and archival NOEMA observations.Our deep NIRSpec/G395M spectroscopy reveals unambiguous broad Balmer emission (FWHM $=2221\pm20$ km s $^{-1}$ ), confirming a super-Eddington accreting black hole ( $\lambda_{\rm Edd}=2.7\pm0.4$ ) with a mass of $\log(M_{\rm BH}/M_{\odot})=7.55\pm0.34$ , using accretion-rate corrected BH mass estimators.After subtracting the point source, we robustly detect stellar emission from the host galaxy across multiple NIRCam and MIRI filters.Our joint morphological–spectral analysis yields a stellar mass of $\log(M_{\ast}/M_\odot)=10.5\pm0.4$ and an intense star formation rate of ${\rm SFR}=330\pm97 M_\odot \mathrm{yr^{-1}}$ , confirming the host as a massive, dusty starburst galaxy.We find that $\targ$ lies on the local $M_{\rm BH}$ – $M_\ast$ relation ( $M_{\rm BH}/M_\ast \simeq 0.001$ ) and is well positioned to evolve into the locus of massive SDSS quasars with $\log(M_{\rm BH}/M_\odot)\approx9$ and $M_\ast\approx10^{11} M_\odot$ at $z\sim6$ , owing to its remarkably rapid growth in both the black hole and its host galaxy.This stands in stark contrast to many recently reported JWST AGN populations at similar redshifts, including the little red dots (LRDs), whose weak or undetected star formation makes it difficult for them to grow into the massive galaxies hosting SDSS-like quasars.These results suggest that $\targ$ marks a rare, pivotal phase of early BH–galaxy co-evolution, plausibly providing a crucial direct pathway to the supermassive black hole systems within the first billion years of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2602.12325/./figures/01_MIRI_LRS_fit.png" alt="Fig12" width="100%"/>

**Figure 12. -** The observed rest-frame optical-to-NIR spectrum, generated from the MIRI LRS observation.
    _Top:_ 2D spectrum from the MIRI LRS observation.
    _Middle:_ 1D spectrum extracted from the 2D spectrum and the corresponding model.
    Black and red solid lines represent the observed spectrum and the resultant best-fit model, respectively.
    Component modeling includes a power-law continuum (blue dashed line), $\ha$ emission (orange dashed line), and decoupled narrow (green) and broad (purple) emission line profiles.
    The corresponding emission lines are labeled with different colors (purple for broad lines and green for narrow lines).
    Note that we do not consider the instrumental broadening during the fitting, the broadening of those rest-frame NIR lines can be caused by the instrument.
    _Bottom:_ The residual between the data and the model. The 1$\sigma$ error is shown as the gray shaded region.  (*fig1.1: LRS*)

</div>
<div id="div_fig2">

<img src="tmp_2602.12325/./figures/03_SED_fitting.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Best-fit multi-component SED decomposition obtained using $\cigale$.
    The observed photometry is shown as red points, comprising both the new JWST observations and archival data \citep{Fujimoto+2022}.
    Dashed lines indicate the individual model components, including stellar population (blue), nebular emission (green), AGN (red), and cold dust (yellow).
    The solid black line represents the total integrated emission model.
     (*fig3: sed*)

</div>
<div id="div_fig3">

<img src="tmp_2602.12325/./figures/PaLineMap.png" alt="Fig6.1" width="50%"/><img src="tmp_2602.12325/./figures/Paa_fit_1Gauss_zoom.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** _ Left:_ The line intensity map generated from the MIRI MRS data around the Pa$\alpha$ emission, with a bandwidth of 1000 $\rm km s^{-1}$.
    The lightblue cross in the panel presents the locus of the quasar, and the dashed circle represents an aperture with a radius of 0$\farcs$5.
    _ Right:_ Spectrum extracted from the central $0$\farcs$5$ aperture of the MIRI observation.
    Upper panel: The observed flux (solid black line) and associated error (grey shaded region), compared against the best-fit continuum and emission model (green line).
    Lower panel: The residuals derived from subtracting the model from the observed data. (*fig4:miri*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.12325"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  A Protoplanet Candidate in the PDS 66 Disk Indicated by Silicon Sulfide Isotopologues 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.12522-b31b1b.svg)](https://arxiv.org/abs/2602.12522)<mark>Appeared on: 2026-02-16</mark> -  _16 pages, 8 figures, accepted for publication in ApJL_

</div>
<div id="authors">

T. C. Yoshida, et al. -- incl., <mark>M. Benisty</mark>, <mark>K. Doi</mark>

</div>
<div id="abstract">

**Abstract:** Despite observational progress in planet formation, the stage in which planetesimals grow into planets remains poorly understood.During this phase, protoplanets may develop gaseous envelopes that are warmer than the surrounding disk gas, potentially providing observable signatures through molecules otherwise depleted in cold regions.In this Letter, we report the detection of the silicon sulfide isotopologues $\isis$ and $\jsis$ in the protoplanetary disk around PDS 66 (MP Mus) at a significance of ${\sim}5{-}6\sigma$ , using the Atacama Large Millimeter/submillimeter Array.These constitute the second and first detections of $\rm ^{28}SiS$ and $\rm ^{30}SiS$ in a protoplanetary disk, respectively.The emission appears as a compact source at $r=60$ au in the southwestern region of the disk, unresolved with a ${\sim}0\farcs5$ beam, and shows a velocity consistent with Keplerian rotation, suggesting a protoplanetary origin.By modeling the line fluxes, we constrain the emitting radius to ${\sim}0.5{-}4$ au and estimate an SiS mass of $10^{22}{-}10^{23}$ g, corresponding to at least ${\sim}10\%$ of the silicon contained in local dust grains.Because complete sublimation of a substantial fraction of dust grains by local processes is difficult to achieve, this result instead implies an accumulation of silicon from a larger region.We propose that a circumplanetary envelope surrounding a low-mass protoplanet, where pebble accretion and subsequent sublimation of grains may enhance gaseus silicon abundance with respect to observable dust grains around it, can account for the observed characteristics.

</div>

<div id="div_fig1">

<img src="tmp_2602.12522/./mom0.png" alt="Fig7" width="100%"/>

**Figure 7. -** Upper panels: 338 GHz continuum image \citep{2025ApJ...984L...9C} and $Q_{\phi}$ image in the H band \citep{2018ApJ...863...44A}. Bottom panels: S/N maps of the \isis and \jsis lines at channels that cover $v_{\rm LSR} \simeq 2.5-7.5 \kms$ and $3.1-4.4 \kms$, where the emission peaks. Note that there might be uncertainty in the rest frequency adopted here (Appendix \ref{app:freq}).
In all panels, the dotted and dashed ellipses indicate radii of $r=60$ and $70$ au, corresponding roughly to the edge of the mm-continuum disk \citep{2023A&A...673A..77R, 2025ApJ...984L...9C}. Contours show the 3 and $6\sigma$ levels of the \jsis line, and the apparent shift between two lines can be explained by statistical error of astrometry (Section \ref{sec:res1}).  The star symbol marks the position of the central star, and the gray ellipse in the lower left corner represents the synthesized beam (except for the H-band image). (*fig:mom0*)

</div>
<div id="div_fig2">

<img src="tmp_2602.12522/./spectra_CDMS.png" alt="Fig2" width="100%"/>

**Figure 2. -** Spectra of the \isis and \jsis lines extracted at the peak S/N positions (Figure \ref{fig:mom0}). The top and bottom panels show spectra from the narrow channel-width cubes, while the middle panel corresponds to the wide-channel \isis cube. The shaded regions indicate $1\sigma$ noise levels. The vertical dashed line marks the systemic velocity, and the green band represents the range of projected Keplerian velocities within one beam. (*fig:spec*)

</div>
<div id="div_fig3">

<img src="tmp_2602.12522/./NRM_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Derived SiS column densities ($N_{\rm SiS}$) and emitting radii ($R$) as a function of the assumed temperature. The bottom panel shows the SiS mass ($M_{\rm SiS}$). For the middle panel ($R$), we also plot the planet's mass assuming that the emitting radius equals to the Hill radius (see Section \ref{sec:pa}).  (*fig:quantities*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.12522"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feh}{\ensuremath{{\rm[Fe/H]}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\lightkurve}{\texttt{lightkurve}}$
$\newcommand{\astrocut}{\texttt{astrocut}}$
$\newcommand{\astroquery}{\texttt{astrocut}}$
$\newcommand{\tesseract}{\texttt{tesseract}}$
$\newcommand{\juliet}{\texttt{juliet}}$
$\newcommand{\zaspe}{\texttt{ZASPE}}$
$\newcommand{\ceres}{\texttt{CERES}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\kms}{\ensuremath{{\rm km s^{-1}}}}$
$\newcommand{\mjup}{\ensuremath{{\rm M_{J}}}}$
$\newcommand{\mearth}{\ensuremath{{\rm M}_{\oplus}}}$
$\newcommand{\mplanet}{\ensuremath{{\rm M_P}}}$
$\newcommand{\rjup}{\ensuremath{{\rm R_J}}}$
$\newcommand{\rplanet}{\ensuremath{{\rm R_P}}}$
$\newcommand{\rstar}{\ensuremath{{\rm R}_{\star}}}$
$\newcommand{\mstar}{\ensuremath{{\rm M}_{\star}}}$
$\newcommand{\lstar}{\ensuremath{{\rm L}_{\star}}}$
$\newcommand{\rsun}{\ensuremath{{\rm R}_{\odot}}}$
$\newcommand{\msun}{\ensuremath{{\rm M}_{\odot}}}$
$\newcommand{\lsun}{\ensuremath{{\rm L}_{\odot}}}$
$\newcommand{\rhoplanet}{\ensuremath{{\rm \rho_P}}}$
$\newcommand{\gccm}{\ensuremath{\mathrm{g} \mathrm{cm}^{-3}}}$
$\newcommand{\mstA}{\ensuremath{1.386_{-0.071}^{+0.071}}}$
$\newcommand{\rstA}{\ensuremath{1.856_{-0.079}^{+0.079}}}$
$\newcommand{\rhostA}{\ensuremath{306_{-39}^{+46}}}$
$\newcommand{\lstA}{\ensuremath{4.7_{-0.2}^{+0.2}}}$
$\newcommand{\ageA}{\ensuremath{2.8_{-0.6}^{+0.6}}}$
$\newcommand{\AvA}{\ensuremath{0.08_{-0.05}^{+0.05}}}$
$\newcommand{\stname}{TIC65910228}$
$\newcommand{\teffA}{\ensuremath{6235.0_{-175}^{+175}}}$
$\newcommand{\fehA}{\ensuremath{0.14_{-0.05}^{+0.05}}}$
$\newcommand{\loggA}{\ensuremath{4.043_{-0.02}^{+0.02}}}$
$\newcommand{\vsiniA}{\ensuremath{8.7_{-0.5}^{+0.5}}}$
$\newcommand{\plname}{TIC65910228b}$
$\newcommand{\aA}{\ensuremath{0.697_{-0.012}^{+0.011}}}$
$\newcommand{\mplA}{\ensuremath{4.554_{-0.247}^{+0.255}}}$
$\newcommand{\rplA}{\ensuremath{1.088_{-0.057}^{+0.061}}}$
$\newcommand{\rhoplA}{\ensuremath{4.374_{-0.674}^{+0.800}}}$
$\newcommand{\teqA}{\ensuremath{582_{-8}^{+8}}}$
$\newcommand{\rprs}{\ensuremath{0.0602_{0.0019}^{0.0022}}}$
$\newcommand{\plbA}{\ensuremath{0.8135_{0.0260}^{0.0200}}}$
$\newcommand{\plaR}{\ensuremath{80.812_{0.698}^{0.680}}}$
$\newcommand{\plinc}{\ensuremath{89.53_{0.01}^{0.01}}}$
$\newcommand{\tdur}{\ensuremath{11.61_{0.41}^{0.47}}}$
$\newcommand{\gplA}{\ensuremath{95.1_{10.4}^{11.8}}}$
$\newcommand{\plecc}{\ensuremath{0.25_{-0.04}^{+0.03}}}$
$\newcommand{\plperA}{\ensuremath{180.524140_{-0.001048}^{+0.000978}}}$
$\newcommand{\rhopost}{\ensuremath{306.34_{-7.87}^{+7.81}}}$
$\newcommand{\plephem}{\ensuremath{2459209.229716_{-0.002220}^{+0.002145}}}$
$\newcommand{\plruno}{\ensuremath{0.8757_{-0.0173}^{+0.0133}}}$
$\newcommand{\plrdos}{\ensuremath{0.0602_{-0.0022}^{+0.0019}}}$
$\newcommand{\plK}{\ensuremath{131.833_{-5.866}^{+5.653}}}$
$\newcommand{\plomega}{\ensuremath{279.13_{-2.63}^{+3.31}}}$
$\newcommand{\mfluxtess}{\ensuremath{196_{327}^{+317} \times 10^{-7}}}$
$\newcommand{\sigmatess}{\ensuremath{2_{-2}^{+13}}}$
$\newcommand{\qunotess}{\ensuremath{0.519_{-0.165}^{+0.248}}}$
$\newcommand{\qdostess}{\ensuremath{0.406_{-0.274}^{+0.353}}}$
$\newcommand{\gpsigmatess}{\ensuremath{100_{-19}^{+29} \times 10^{-6}}}$
$\newcommand{\gprhotess}{\ensuremath{1.209_{-0.335}^{+0.503}}}$
$\newcommand{\mfluxhatpi}{\ensuremath{46_{223}^{+233} \times 10^{-7}}}$
$\newcommand{\sigmahatpi}{\ensuremath{0_{-0}^{+12}}}$
$\newcommand{\qunohatpi}{\ensuremath{0.276_{-0.200}^{+0.337}}}$
$\newcommand{\qdoshatpi}{\ensuremath{0.431_{-0.308}^{+0.352}}}$
$\newcommand{\diluthatpi}{\ensuremath{0.904_{-0.066}^{+0.062}}}$
$\newcommand{\mfluxomes}{\ensuremath{2327_{1663}^{+1671} \times 10^{-7}}}$
$\newcommand{\sigmaomes}{\ensuremath{1_{-1}^{+180}}}$
$\newcommand{\qunoomes}{\ensuremath{0.574_{-0.279}^{+0.265}}}$
$\newcommand{\qdosomes}{\ensuremath{0.406_{-0.291}^{+0.315}}}$
$\newcommand{\muferos}{\ensuremath{20027_{8}^{+7}}}$
$\newcommand{\sigmaferos}{\ensuremath{7_{-6}^{+17}}}$
$\newcommand{\mupspec}{\ensuremath{20063_{-4}^{+4}}}$
$\newcommand{\sigmapspec}{\ensuremath{19_{-4}^{+4}}}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# TIC65910228b: A single-transit discovery of a massive long-period warm Jupiter with TESS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.13125-b31b1b.svg)](https://arxiv.org/abs/2602.13125)<mark>Appeared on: 2026-02-16</mark> -  _11 pages, 11 figures_

</div>
<div id="authors">

F. I. Rojas, et al. -- incl., <mark>L. Acuña</mark>, <mark>T. Henning</mark>, <mark>Y. Reinarz</mark>

</div>
<div id="abstract">

**Abstract:** Warm Jupiters are excellent case studies for the investigation of giant planet internal structures and formation theories. However, the sample of long-period transiting giants is still small today for a better understanding of this population. Starting from a single transit found in the Transiting Exoplanet Survey Satellite (TESS) data, we confirm the planetary nature of the signal and measure its orbital parameters, mass, and radius. We put this system in the context of long-period giant transiting planets and analyzed the viability to sustain atmospheric or dynamical follow-up. We carried out a spectroscopic follow-up using FEROS and PLATOSpec to obtain precise radial velocities. We added a photometric follow-up with HATPI and Observatoire Moana to obtain a more precise estimate of the orbital period. We derived the orbital and physical parameters through a joint analysis of this data. We report the discovery and characterization of $\plname$ , a transiting warm Jupiter with a mass of $4.554\pm0.255$ $\mjup$ and a radius of $1.088\pm0.061$ $\rjup$ , orbiting an evolved F-type star every $\sim180.52$ days in an eccentric orbit ( $e=0.25\pm 0.04$ ). This planet joins a still under-explored population of long-period ( $P>100$ ) massive ( $M_p > 4$ $\mjup$ ) transiting giant planets, being one of the few with a mild eccentricity. This target is a nice example of the potential of single-transit events to populate this region of the parameter space.

</div>

<div id="div_fig1">

<img src="tmp_2602.13125/./figs/LCs.png" alt="Fig7" width="100%"/>

**Figure 7. -** Top panel: Observed transits with TESS, HATPI and OM-ES1, respectively. Model from joint-fit is overplotted. White circles correspond to 1-hour bins in HATPI and OMES1 light curves. Bottom panel: Residuals after removing the transit model. (*fig:LCs*)

</div>
<div id="div_fig2">

<img src="tmp_2602.13125/./figs/Pvse.png" alt="Fig10.1" width="50%"/><img src="tmp_2602.13125/./figs/PvsR.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Planets with $P>10$ days and their orbital period vs eccentricity and planetary radii. The left panel shows the dependency of orbital period on eccentricity. $\log g$ is plotted in color scale. The right panel shows the period-radius diagram with $\plname$ being one of the longer period giant planets discovered. (*fig:PvseR*)

</div>
<div id="div_fig3">

<img src="tmp_2602.13125/./figs/Mvsmet.png" alt="Fig11.1" width="50%"/><img src="tmp_2602.13125/./figs/metvse.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Left: Mass vs stellar metallicity diagram for planets with $P>10$ d. Kernel density estimation for planets with mass lower and higher than $0.1$\mjup are plotted for reference. Right: Metallicity vs eccentricity diagram for planets with $P>10$ d and $M>0.1$\mjup. $\plname$ is shown in green in both panels. (*fig:Mvsmet*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.13125"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

125  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
